In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sympy import latex
from IPython.display import display, Math
plt.rcParams['figure.figsize'] = [20, 10]

In [3]:
# retrived from https://www.arduino.cc/reference/en/language/functions/math/map/
def map_func(x, in_min, in_max, out_min, out_max):
    return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min

In [4]:
def adc2angle(df, shoulder_min=910, soulder_max=132, elbow_min=145, elbow_max=820, elbow_min_angle=160):
    col = df.columns # extract column names from dataframe
    
    # convertes first column to angles between -90 and 90
    df[col[0]] = df.apply(lambda x: map_func(df[col[0]], shoulder_min, soulder_max, -90, 90)) 
    
    # convertes second column to angles between 0 and 160
    df[col[1]] = df.apply(lambda x: map_func(df[col[1]], elbow_min, elbow_max, 0, elbow_min_angle))
    return df

# Mapping Calibration

In [5]:
df = pd.read_csv('second_trial_2.csv') # dataset generated during calibration run
df # prints pot ADC values

,Sholder Angle,Elbow Angle
0,907.0,144.0
1,907.0,144.0
2,906.0,144.0
3,913.0,817.0
4,914.0,818.0
5,914.0,818.0
6,132.0,147.0
7,132.0,147.0
8,132.0,147.0


In [6]:
df['Sholder Angle'] = df.apply(lambda x: map_func(df['Sholder Angle'], 910, 132, -90, 90))
df['Elbow Angle'] = df.apply(lambda x: map_func(df['Elbow Angle'], 145, 820, 0, 160))
df # prints pot converted angle values

,Sholder Angle,Elbow Angle
0,-89.305913,-0.237037
1,-89.305913,-0.237037
2,-89.074550,-0.237037
3,-90.694087,159.288889
4,-90.925450,159.525926
5,-90.925450,159.525926
6,90.000000,0.474074
7,90.000000,0.474074
8,90.000000,0.474074


# Converting ADC Values of Traced Image to Angles

In [7]:
df = pd.read_csv('second_trial_3.csv') # second_trial_3.csv is the results of the traced image
df = adc2angle(df)
df.to_csv('angles.csv', index=False)